In [1]:
import math

import sys

import os
import numpy as np
import pandas as pd

from astropy.io import fits
from astropy.table import Table

import matplotlib
from matplotlib import pyplot as plt
# %matplotlib inline

sys.path.insert(0, "/global/homes/c/cinlima/gcr-catalogs")
sys.path.insert(0, "global/homes/c/cinlima/clevar/clevar")

import GCRCatalogs
#Put nerc if using nersc
GCRCatalogs.set_root_dir_by_site("nersc")

from clevar import ClCatalog, MemCatalog
from clevar.match import MembershipMatch, get_matched_pairs

from clevar.match import output_matched_catalog
import clevar
clevar.__version__


# matplotlib.rcParams.update({'font.size': 22})
# matplotlib.rcParams.update({'figure.figsize': (10,8)})

sys.path.insert(0, 'NumCosmo/notebooks/richness_proxy/CDC2Data/')


In [2]:
clevar.__version__

#GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_image').get_catalog_info()


'0.14.1'

In [3]:
min_richness = 5
min_halo_mass = 1e13 #Msun
redshift_max = 1.2

# gc = GCRCatalogs.load_catalog('skysim5000_v1.1.1_redmapper_v0.8.5')
# gc_truth = GCRCatalogs.load_catalog('skysim5000_v1.1.1_image')  

gc = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_redmapper_v0.8.1')
gc_truth = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_image')  
    
quantities = gc.list_all_quantities()
cluster_quantities = [q for q in quantities if 'member' not in q]
member_quantities = [q for q in quantities if 'member' in q]
    
query = GCRCatalogs.GCRQuery('(richness > ' + str(min_richness) +')')
cluster_data = Table(gc.get_quantities(cluster_quantities, [query]))
member_data = Table(gc.get_quantities(member_quantities))
   
quantities_wanted = ['redshift','halo_mass','halo_id','galaxy_id','ra','dec', 'is_central', 'mag_true_u_lsst', 'mag_true_g_lsst', 'mag_true_r_lsst', 'mag_true_i_lsst', 'mag_true_z_lsst', 'mag_true_y_lsst']
#quantities_wanted = ['redshift','halo_mass','halo_id','galaxy_id','ra','dec', 'is_central']

query = GCRCatalogs.GCRQuery('(halo_mass > ' + str(min_halo_mass) +') & (redshift < ' + str(redshift_max) +')')
truth_data = Table(gc_truth.get_quantities(quantities_wanted, [query]))


In [4]:
inicio = time.time()

RM_cat_name = 'cosmoDC2_v1.1.4_redmapper_v0.8.1'
DC2_cat_name = 'cosmoDC2_v1.1.4_image'

#Redmapper
RM_cat = ClCatalog(RM_cat_name, z=cluster_data["redshift"], ra=cluster_data["ra"], dec=cluster_data["dec"], id = cluster_data["cluster_id"], richness = cluster_data["richness"], richness_err=cluster_data["richness_err"], \
                   id_cg = cluster_data["id_cen_0"], ztrue_cg=cluster_data["redshift_true_cg"], p_cg_0=cluster_data["p_cen_0"])

pmem = member_data["p_member"] * member_data["pfree_member"] * member_data["theta_i_member"] * member_data["theta_r_member"] 

member_data.add_column(pmem, name="pmem")

RM_cat_members = MemCatalog("RM_members", id_cluster=member_data["cluster_id_member"], id=member_data["id_member"], ra=member_data["ra_member"], dec=member_data["dec_member"],
                            pmem=member_data["pmem"],  redshift=member_data["redshift_true_member"], 
                            mag_g_lsst_member=member_data["mag_g_lsst_member"], mag_r_lsst_member=member_data["mag_r_lsst_member"], 
                            mag_i_lsst_member=member_data["mag_i_lsst_member"], mag_z_lsst_member=member_data["mag_z_lsst_member"], mag_y_lsst_member=member_data["mag_y_lsst_member"]) 
RM_cat.add_members(members_catalog=RM_cat_members)


#Dc2
halo_cat = ClCatalog(DC2_cat_name, z= truth_data["redshift"][truth_data["is_central"]],
                     ra= truth_data["ra"][truth_data["is_central"]], dec= truth_data["dec"][truth_data["is_central"]],
                     id=truth_data["halo_id"][truth_data["is_central"]], mass_fof=truth_data["halo_mass"][truth_data["is_central"]])

halo_cat_members = MemCatalog("halo_members", id_cluster=truth_data["halo_id"], id=truth_data["galaxy_id"], ra=truth_data["ra"], dec=truth_data["dec"],
                              redshift=truth_data["redshift"], is_central = truth_data["is_central"], 
                              mag_true_u_lsst = truth_data["mag_true_u_lsst"],  mag_true_g_lsst = truth_data["mag_true_g_lsst"],
                              mag_true_r_lsst = truth_data["mag_true_r_lsst"], mag_true_i_lsst = truth_data["mag_true_i_lsst"],  mag_true_z_lsst = truth_data["mag_true_z_lsst"], ) 

halo_cat.add_members(members_catalog=halo_cat_members)

RM_cat._init_match_vals(True)
halo_cat._init_match_vals(True)

#Matching
mt = MembershipMatch()
match_config = {
    'type': 'cross', # options are cross, cat1, cat2
    'preference': 'shared_member_fraction', # other options are more_massive, angular_proximity or redshift_proximity
    'minimum_share_fraction': 0,
    'match_members_kwargs': {'method':'id'},
}
mt.match_from_config(RM_cat, halo_cat, match_config)


fim = time.time()
tempo_total = fim - inicio

print(f"Tempo de execução: {tempo_total/60:.3}min")

NameError: name 'time' is not defined

In [4]:
RM_cat_name = 'skysim5000_v1.1.1_redmapper_v0.8.5'
DC2_cat_name = 'skysim5000_v1.1.1_image'

# RM_cat_name = 'cosmoDC2_v1.1.4_redmapper_v0.8.1'
# DC2_cat_name = 'cosmoDC2_v1.1.4_image'

RM_cat = ClCatalog(RM_cat_name, z=cluster_data["redshift"], ra=cluster_data["ra"], dec=cluster_data["dec"], id = cluster_data["cluster_id"], richness = cluster_data["richness"], richness_err=cluster_data["richness_err"], \
                   id_cg = cluster_data["id_cen_0"], ztrue_cg=cluster_data["redshift_true_cg"], p_cg_0=cluster_data["p_cen_0"])



id,z,ra,dec,richness,richness_err,id_cg,ztrue_cg,p_cg_0,mt_self,mt_other,mt_multi_self,mt_multi_other
str11,float32,float64,float64,float32,float32,int64,float32,float32,object,object,object,object
34,0.21707904,4.801543436336212,-3.430296893154116,248.12637,3.64447,6466000014572,0.21771926,0.9986252,None,None,[],[]
136,0.3411566,4.724775814119802,-0.4457212024208109,185.82506,3.4095232,6081000026524,0.33966494,0.98930895,None,None,[],[]
142,0.5101348,3.734412956523043,-0.7502042775962834,163.7206,2.5087166,6081000093184,0.51063555,0.9998284,None,None,[],[]
1365,0.5912586,2.896798269949837,-1.5203787028802902,87.18639,2.3910992,6209000243030,0.59068364,0.7898552,None,None,[],[]
1510,0.42078963,4.760800620903602,-3.4031714124970533,88.62036,2.40914,6466000092913,0.4168227,0.99990094,None,None,[],[]
1549,0.5069738,4.290126649178689,-0.16020782034271513,83.693,2.3135304,6081000094492,0.50784713,0.9911676,None,None,[],[]
1664,0.14206502,6.015723330476854,-3.333165464623904,74.540016,1.9305084,6466000003732,0.14274381,0.99998873,None,None,[],[]
1674,0.5376342,4.779452073850936,-0.1736404654609399,74.753586,2.1025321,6081000089361,0.5307513,0.9999747,None,None,[],[]
1777,0.6139372,5.3226312459458365,-3.816390369189824,80.550476,2.2452028,6466000244571,0.61191887,0.9979278,None,None,[],[]


In [5]:
# display(RM_cat)

In [6]:
RM_cat.write('RM_cat_richness_min_'+str(min_richness)+'.fits', add_header =True, overwrite=True)
#RM_cat.write('NumCosmo/notebooks/richness_proxy/RM_cat_richness_min_'+str(min_richness)+'.fits', add_header =True, overwrite=True)

In [7]:
pmem = member_data["p_member"] * member_data["pfree_member"] * member_data["theta_i_member"] * member_data["theta_r_member"] 

member_data.add_column(pmem, name="pmem")

In [8]:
RM_cat_members = MemCatalog("RM_members", id_cluster=member_data["cluster_id_member"], id=member_data["id_member"], ra=member_data["ra_member"], dec=member_data["dec_member"],
                            pmem=member_data["pmem"],  redshift=member_data["redshift_true_member"], 
                            mag_g_lsst_member=member_data["mag_g_lsst_member"], mag_r_lsst_member=member_data["mag_r_lsst_member"], 
                            mag_i_lsst_member=member_data["mag_i_lsst_member"], mag_z_lsst_member=member_data["mag_z_lsst_member"], mag_y_lsst_member=member_data["mag_y_lsst_member"]) 
RM_cat.add_members(members_catalog=RM_cat_members)

# #RM_cat.members.write('NumCosmo/notebooks/richness_proxy/CDC2Data/RM_cat_richness_min_'+str(min_richness)+'_members.fits', overwrite=True)
# RM_cat.members.write('RM_cat_richness_min_'+str(min_richness)+'_members.fits', overwrite=True)


In [9]:
halo_cat = ClCatalog(DC2_cat_name, z= truth_data["redshift"][truth_data["is_central"]],
                     ra= truth_data["ra"][truth_data["is_central"]], dec= truth_data["dec"][truth_data["is_central"]],
                     id=truth_data["halo_id"][truth_data["is_central"]], mass_fof=truth_data["halo_mass"][truth_data["is_central"]])
halo_cat

id,z,ra,dec,mass_fof,mt_self,mt_other,mt_multi_self,mt_multi_other
str21,float64,float64,float64,float64,object,object,object,object
4010066475,0.04554527819345999,51.008516490175516,-40.83107772618864,33506664194163.383,None,None,[],[]
8410066475,0.048258542345124944,50.95764333668338,-41.34449214014141,74621860826501.4,None,None,[],[]
13910066475,0.0481862270842659,49.638658115154634,-40.400678302137855,16891355712901.41,None,None,[],[]
710066464,0.06438326462902522,50.57650523362703,-41.15895234883735,17911434075583.098,None,None,[],[]
31710066442,0.1137579883816715,49.504804200029895,-40.23973139541635,69213477135143.664,None,None,[],[]
42110066442,0.1259633143787311,51.069670404722665,-39.645283542113106,62974036412214.086,None,None,[],[]
57710066442,0.12743589142235479,50.60532208979055,-39.33595810866306,48388878878287.33,None,None,[],[]
71610066442,0.10736840257999059,50.77613148927426,-39.16862215638887,13042641686354.93,None,None,[],[]
7910066432,0.12775858757827474,50.36982737040335,-39.097459789992094,34125997485791.55,None,None,[],[]


In [10]:
#halo_cat.write('NumCosmo/notebooks/richness_proxy/CDC2Data/halo_cat_mass_min_'+ f"{min_halo_mass:1.2e}"+'.fits', overwrite=True)
# halo_cat.write('halo_cat_mass_min_'+ f"{min_halo_mass:1.2e}"+'.fits', overwrite=True)


In [11]:
halo_cat_members = MemCatalog("halo_members", id_cluster=truth_data["halo_id"], id=truth_data["galaxy_id"], ra=truth_data["ra"], dec=truth_data["dec"],
                              redshift=truth_data["redshift"], is_central = truth_data["is_central"], 
                              mag_true_u_lsst = truth_data["mag_true_u_lsst"],  mag_true_g_lsst = truth_data["mag_true_g_lsst"],
                              mag_true_r_lsst = truth_data["mag_true_r_lsst"], mag_true_i_lsst = truth_data["mag_true_i_lsst"],  mag_true_z_lsst = truth_data["mag_true_z_lsst"], ) 


In [12]:
halo_cat_members

id,id_cluster,ra,dec,redshift,is_central,mag_true_u_lsst,mag_true_g_lsst,mag_true_r_lsst,mag_true_i_lsst,mag_true_z_lsst
str21,str21,float64,float64,float64,bool,float32,float32,float32,float32,float32
10066000000072,4010066475,51.008516490175516,-40.83107772618864,0.04554527819345999,True,16.616812,14.937185,14.107611,13.733477,13.46904
10066000000073,4010066475,51.025154929742484,-40.845821579233935,0.046072561671865575,False,20.1658,18.890131,18.359976,18.100388,17.965904
10066000000074,4010066475,50.72472382760179,-40.57478437526675,0.045541266862879404,False,16.636957,14.957333,14.127756,13.753624,13.489187
10066000000075,4010066475,50.81571008109443,-40.59157963288105,0.04531300720087894,False,21.162346,19.918613,19.267712,18.875559,18.552221
10066000000076,4010066475,51.0590243314875,-40.83909415345591,0.04547781829126207,False,21.695549,20.191692,19.435575,19.067238,18.837688
10066000000077,4010066475,51.046995498068696,-40.81164729369869,0.04510798075102729,False,17.266966,15.784197,15.054684,14.694525,14.458246
10066000000078,4010066475,51.066331267759075,-40.88275021954884,0.04658017672825321,False,22.088558,20.706915,20.059488,19.755507,19.591475
10066000000079,4010066475,51.10025645305562,-40.92527470689328,0.04349527948965948,False,22.487223,20.992882,20.250214,19.893808,19.673271
10066000000080,4010066475,51.08218725961258,-40.98542072893441,0.04576709666611212,False,22.175161,20.835583,20.19861,19.897142,19.75168


In [13]:
halo_cat.add_members(members_catalog=halo_cat_members)


/global/homes/c/cinlima/.local/lib/python3.11/site-packages/clevar-0.14.1-py3.11.egg/clevar/catalog/catalog.py:728: UserWarning: Some galaxies were not members of the cluster catalog. They are stored in leftover_members attribute.
  warnings.warn(


In [14]:
#halo_cat.members.write('NumCosmo/notebooks/richness_proxy/CDC2Data/halo_cat_mass_min_'+f"{min_halo_mass:1.2e}"+'_members.fits', overwrite=True)
# halo_cat.members.write('halo_cat_mass_min_'+f"{min_halo_mass:1.2e}"+'_members.fits', overwrite=True)


In [15]:
# RM_cat = ClCatalog.read('RM_cat_richness_min_'+str(min_richness)+'.fits', name='RM_cat', full=True, tags={'mass':'richness'})
# halo_cat = ClCatalog.read('halo_cat_mass_min_'+f"{min_halo_mass:1.2e}"+'.fits', name='halo_cat', full=True, tags={'mass':'mass_fof'})



In [16]:
# RM_cat.read_members('RM_cat_richness_min_'+str(min_richness)+'_members.fits',  full=True)
# halo_cat.read_members('halo_cat_mass_min_'+f"{min_halo_mass:1.2e}"+'_members.fits', full=True)


In [17]:
RM_cat._init_match_vals(True)
halo_cat._init_match_vals(True)

In [18]:
mt = MembershipMatch()
match_config = {
    'type': 'cross', # options are cross, cat1, cat2
    'preference': 'shared_member_fraction', # other options are more_massive, angular_proximity or redshift_proximity
    'minimum_share_fraction': 0,
    'match_members_kwargs': {'method':'id'},
}
mt.match_from_config(RM_cat, halo_cat, match_config)


308,136 members were matched.


KeyError: 'mass'

In [ ]:
#mt.save_matches(RM_cat, halo_cat, out_dir='NumCosmo/notebooks/richness_proxy/CDC2Data/cosmoDC2_RM_match_cross_membership', overwrite=True)
# mt.save_matches(RM_cat, halo_cat, out_dir='SS_RM_match_cross_membership', overwrite=True)


In [ ]:
#mt.load_matches(RM_cat, halo_cat, out_dir='NumCosmo/notebooks/richness_proxy/CDC2Data/cosmoDC2_RM_match_cross_membership')
# mt.load_matches(RM_cat, halo_cat, out_dir='SS_RM_match_cross_membership')


In [ ]:
RM_cat_bij, halo_cat_bij = get_matched_pairs(RM_cat, halo_cat, 'cross')

In [ ]:
RM_cat_bij.size, RM_cat_bij.size/RM_cat.size,  halo_cat_bij.size/halo_cat.size


In [ ]:
plt.scatter(np.log(halo_cat_bij['mass_fof']),np.log(RM_cat_bij['richness']))
# plt.scatter(halo_cat_bij['mass_fof'],RM_cat_bij['richness'])

plt.xlabel('$\ln (M_{fof}$)')
plt.ylabel('$\ln (\lambda)$')

In [ ]:
Table([halo_cat_bij['mass_fof'], RM_cat_bij['richness'], RM_cat_bij['z']])